## Colin's active development sandbox

In [34]:
from enum import Enum
from pathlib import Path
import logging
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision as tv
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import tqdm.auto as tqdm
import os
import re

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] - [%(levelname)s]: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

class DataPaths(Enum):
    ROOT_DATA_DIR = str(Path.cwd().parent / 'data' / 'SoccerNet' / 'jersey-2023' / 'extracted')
    TEST_DATA_DIR = str(Path(ROOT_DATA_DIR) / 'test' / 'images')
    TRAIN_DATA_DIR = str(Path(ROOT_DATA_DIR) / 'train' / 'images')
    VALIDATION_DATA_DIR = str(Path(ROOT_DATA_DIR) / 'challenge' / 'images')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f"ROOT_DATA_DIR: {DataPaths.ROOT_DATA_DIR.value}")
logging.info(f"TRAIN_DATA_DIR: {DataPaths.TRAIN_DATA_DIR.value}")
logging.info(f"TEST_DATA_DIR: {DataPaths.TEST_DATA_DIR.value}")
logging.info(f"VAL_DATA_DIR: {DataPaths.VALIDATION_DATA_DIR.value}")
logging.info(f"Using device: {device}")


[2025-02-26 09:40:07] - [INFO]: ROOT_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted
[2025-02-26 09:40:07] - [INFO]: TRAIN_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images
[2025-02-26 09:40:07] - [INFO]: TEST_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images
[2025-02-26 09:40:07] - [INFO]: VAL_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\challenge\images
[2025-02-26 09:40:07] - [INFO]: Using device: cuda


### Tracklet iterator

In [35]:
# Ignore the .DS_Store files
tracks = [t for t in os.listdir(DataPaths.TRAIN_DATA_DIR.value) if not t.startswith('.')]
logging.info(tracks[0:10])

# Extract numerical part and convert to integer for comparison
def extract_number(track):
    match = re.search(r'(\d+)', track)  # Extracts the first sequence of digits
    if match:
        return int(match.group(1))
    return -1  # Provide a default value if no number is found

# Find min and max tracklets based on the extracted number
if tracks:
    min_track = min(tracks, key=extract_number)
    max_track = max(tracks, key=extract_number)

    logging.info(f"Min tracklet: {min_track}")
    logging.info(f"Max tracklet: {max_track}")
else:
    logging.warning("No tracklets found.")

[2025-02-26 09:40:07] - [INFO]: ['0', '1', '10', '100', '1000', '1001', '1002', '1003', '1004', '1005']
[2025-02-26 09:40:07] - [INFO]: Min tracklet: 0
[2025-02-26 09:40:07] - [INFO]: Max tracklet: 1426
